In [1]:
#pip3 install notebook ipywidgets plotly
#jupyter nbextension enable --py widgetsnbextension
#jupyter nbextension enable --py plotlywidget

In [2]:
import igraph as ig
import pandas as pd
import webbrowser
import json
import urllib.request
import chart_studio.plotly as py
import plotly.graph_objs as go
from ipywidgets import widgets

from utils import *

import sys
sys.path.append("..")
from predict import *
from config import *

import os
parent_dir = os.path.dirname(os.getcwd())

This visualization is inspired from plotly officiel tutorials: https://plot.ly/python/v3/3d-network-graph/

In [3]:
#Load data for the graph
df_node = pd.read_csv(os.path.join(parent_dir,DF_NODE_FILENAME)).drop('Unnamed: 0',axis=1)
df_edge = pd.read_csv(os.path.join(parent_dir,DF_EDGE_FILENAME)).drop('Unnamed: 0',axis=1)

In [4]:
df_node.head()

,name,url
0,space research,https://en.wikipedia.org/wiki/space_research
1,space exploration,https://en.wikipedia.org/wiki/space_exploration
2,space race,https://en.wikipedia.org/wiki/space_race
3,space probe,https://en.wikipedia.org/wiki/space_probe
4,u.s. space exploration history on u.s. stamps,https://en.wikipedia.org/wiki/u.s._space_explo...


In [5]:
df_edge.head()

,source,target
0,0,431
1,0,432
2,0,429
3,0,82
4,0,433


In [6]:
# Create placeholders for graph information
labels = df_node['name'].tolist()
color_node = [0.0 for i in range(len(labels))]
color_node_original = color_node.copy()
texts_to_show = [None for i in range(len(labels))]
texts_to_show_original = texts_to_show.copy()
# Create plotting items
# title, annotations, textbox_query, selector, query_answer, g = create_plot_items(df_node,df_edge,labels,color_node,texts_to_show)

In [7]:
# import pickle
# with open('graph,pkl','wb') as f:
#     pickle.dump(g,f)

In [8]:
# Load graph and models
title, annotations, textbox_query, selector, query_answer, g = load_graph('graph,pkl')
bert_embedder_spectral, bert_embedder_mean, node2vec_embed, df_node = load_models(parent_dir)

In [13]:
# node2vec_embed.wv.vocab

In [11]:
current_selector = selector.value

# Choose the model
def response(change):
    global current_selector
    current_selector = selector.value
    make_query(textbox_query)
    
# On a new query, compute the predictions and color the nodes accordingly
def make_query(user_query):
    global current_selector
    global color_node_original
    global texts_to_show_original
    texts_to_show = texts_to_show_original.copy()
    color_node = color_node_original.copy()
    
    query = user_query.value
    if query != "":
        preds = make_prediction(query, current_selector, bert_embedder_spectral, bert_embedder_mean, node2vec_embed, df_node)
        #preds = {10:1,11:0.8,12:0.6,13:0.4,14:0.3}
        if preds == None:
            html_text = create_text([],df_node)
            query_answer.value = html_text
        else:
            dict_colors = compute_color(preds)

            html_text = create_text(list(dict_colors.keys()),df_node)
            query_answer.value = html_text

            for k,v in dict_colors.items():
                color_node[k] = v
                texts_to_show[k] = labels[k]

            with g.batch_update():
                g.data[1].marker.color = color_node
                g.data[1].text = texts_to_show
            
# Open url when clicking on node
def update_point(trace, points, selector):
    if len(points.point_inds) != 0:
        url = g.data[1].customdata[points.point_inds[0]]
        webbrowser.open_new_tab(url)
    
textbox_query.on_submit(make_query)
selector.observe(response, names="value")
g.data[1].on_click(update_point)

In [12]:
container = widgets.HBox(children=[textbox_query, selector])
widgets.VBox([title, annotations, container, query_answer, g])